## Proof-of-concept for dynamically adjusting social clock to sun clock
### Adapted from 'SmoothTime.py' file by Masoud Abdouli
### Author: Masoud Abdouli
### Author: Harini Subramanian (mainly review)


In [3]:
from astral import LocationInfo
from astral.sun import sun
from astral import Observer
import datetime
import time
from tkinter import *
from tkinter.ttk import *


In [4]:
# 1. Setting up 
    # a) the timezone 
timezone = datetime.timedelta(hours = 3, minutes = 30)

    # b) location (latitude, longitude of Tehran, Iran)
city = LocationInfo("Tehran", "Iran", "Asia/Tehran", 35.6892523, 51.3896004)

    # c) observer (Defines the location of an observer on Earth.)
observer = Observer(latitude = 35.6892523, # city.latitude, 
                    longitude = 51.3896004) # city.longitude)

    # d) in UTC
current_utc = datetime.datetime.now().date()
print("Current UTC time: ", current_utc)

    # e) in the current Tehran timezone
current_local = datetime.datetime.utcnow() + timezone
print("Current local time: ", current_local)

Current UTC time:  2025-04-19
Current local time:  2025-04-19 10:51:38.441408


In [5]:
# 2. Storing the time of dawn, sunrise, noon and sunset and dusk in UTC:
    # a) today
c_data = sun(observer, date = current_utc)

    # b) tomorrow
t_data = sun(observer, date = current_utc + datetime.timedelta(days = 1))

    # c) yesterday
y_data = sun(observer, date = current_utc - datetime.timedelta(days = 1))


In [6]:
# 3. Extracting the sunrise, noon and sunset times for today, tomorrow and yesterday in UTC
    # a) Today's sunrise
utc_sunrise = c_data["sunrise"]
#utc_noon = (c_data["noon"])
print('Current UTC sunrise time: ', utc_sunrise)

    # b) Today's sunset
utc_sunset = c_data["sunset"]
print('Today UTC sunset time: ', utc_sunset)

    # c) Tomorrow's sunrise
utc_sunrise_t = t_data["sunrise"]
print('Tomorrow UTC sunset time: ', utc_sunrise_t)

    # d) Yesterday's sunset
utc_sunset_y = y_data["sunset"]
print('Yesterday UTC sunset time: ', utc_sunset_y)

Current UTC sunrise time:  2025-04-19 01:56:49.831457+00:00
Today UTC sunset time:  2025-04-19 15:10:49.136223+00:00
Tomorrow UTC sunset time:  2025-04-20 01:55:34.368985+00:00
Yesterday UTC sunset time:  2025-04-18 15:09:58.992443+00:00


In [7]:
# 4. Calculating the duration of sunlight for today, tomorrow and yesterday in UTC
    # a) Today's sunlight duration
sunlight_duration = (c_data["sunset"])-(c_data["sunrise"])
print('Today UTC sunlight duration: ', sunlight_duration)

    # b) Tomorrow's sunlight duration
sunlight_duration_t = (t_data["sunset"])-(t_data["sunrise"])
print('Tomorrow UTC sunlight duration: ', sunlight_duration_t)

    # c) Yesterday's sunlight duration
sunlight_duration_y = (y_data["sunset"])-(y_data["sunrise"])
print('Yesterday UTC sunlight duration: ', sunlight_duration_y)

Today UTC sunlight duration:  13:13:59.304766
Tomorrow UTC sunlight duration:  13:16:04.964633
Yesterday UTC sunlight duration:  13:11:52.934697


In [10]:
# 5. Calculating sunrise and sunset times as in step 4) to the current timezone
    # a) Today's sunrise time in Tehran timezone
real_sunrise = utc_sunrise + timezone
#real_noon = utc_noon + timezone
print('Today Tehran sunrise time: ', real_sunrise)

    # b) Today's sunset time in Tehran timezone
real_sunset = utc_sunset + timezone
print('Today Tehran sunset time: ', real_sunset)

    # c) Tomorrow's sunrise time in Tehran timezone
real_sunrise_t = utc_sunrise_t + timezone
print('Tomorrow Tehran sunrise time: ', real_sunrise_t)

    # d) Yesterday's sunset time in Tehran timezone
real_sunset_y = utc_sunset_y + timezone
print('Yesterday Tehran sunset time: ', real_sunset_y)

Today Tehran sunrise time:  2025-04-19 05:26:49.831457+00:00
Today Tehran sunset time:  2025-04-19 18:40:49.136223+00:00
Tomorrow Tehran sunrise time:  2025-04-20 05:25:34.368985+00:00
Yesterday Tehran sunset time:  2025-04-18 18:39:58.992443+00:00


In [13]:
# 6. Extracting the time component from the datetime objects
    # a) Exact sunrise time in Tehran
sunrise_time = datetime.datetime.time(real_sunrise)
#noon_time = datetime.datetime.time(real_noon)
print('Exact sunrise time in Tehran: ', sunrise_time)

    # b) Exact sunset time in Tehran
sunset_time = datetime.datetime.time(real_sunset)
print('Exact sunset time in Tehran: ', sunset_time)

    # c) Exact sunrise time in Tehran for tomorrow
sunrise_time_t = datetime.datetime.time(real_sunrise_t)
print('Exact sunrise time in Tehran for tomorrow: ', sunrise_time_t)

    # d) Exact sunset time in Tehran for yesterday
sunset_time_y = datetime.datetime.time(real_sunset_y)
print('Exact sunset time in Tehran for yesterday: ', sunset_time_y)

Exact sunrise time in Tehran:  05:26:49.831457
Exact sunset time in Tehran:  18:40:49.136223
Exact sunrise time in Tehran for tomorrow:  05:25:34.368985
Exact sunset time in Tehran for yesterday:  18:39:58.992443


In [20]:
# 7. Calculating the average noon time
av_noon = datetime.timedelta(hours = 12)
print('Average noon time: ', av_noon)

Average noon time:  12:00:00


In [21]:
# 8. Calculating the polar circle for today
    # a) If the duration of sunlight is less than 3 hours but greater than 1 second, the sunrise and sunset times are calculated as follows:
if datetime.timedelta(seconds = 1) <= sunlight_duration < datetime.timedelta(hours = 3):
        # (i) 'd_prime' is the duration of sunlight doubled
    d_prime = 2 * sunlight_duration
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise = av_noon - (d_prime / 2)
        # (iii) The new sunset time is calculated as the average noon time plus half of the duration of sunlight
    new_sunset = av_noon + (d_prime / 2)

    # b) If the duration of sunlight is less than 9 hours but greater than 3 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 3) <= sunlight_duration < datetime.timedelta(hours = 9):
        # (i) 'd_prime' is the duration of sunlight plus 3 hours
    d_prime = sunlight_duration + datetime.timedelta(hours = 3)
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise = av_noon - (d_prime / 2)
        # (iii) The new sunset time is calculated as the average noon time plus half of the duration of sunlight
    new_sunset = av_noon + (d_prime / 2)

    # c) If the duration of sunlight is less than 15 hours but greater than 9 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 9) <= sunlight_duration < datetime.timedelta(hours = 15):
        # (i) 'd_prime' is 12 hours
    d_prime = datetime.timedelta(hours = 12)
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise = av_noon - (d_prime / 2)
        # (iii) The new sunset time is calculated as the average noon time plus half of the duration of sunlight
    new_sunset = av_noon + (d_prime / 2)

    # d) If the duration of sunlight is less than 21 hours but greater than 15 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 15) <= sunlight_duration < datetime.timedelta(hours = 21):
        # (i) 'd_prime' is the duration of sunlight minus 3 hours
    d_prime = sunlight_duration - datetime.timedelta(hours = 3)
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise = av_noon - (d_prime / 2)
        # (iii) The new sunset time is calculated as the average noon time plus half of the duration of sunlight
    new_sunset = av_noon + (d_prime / 2)

    # e) If the duration of sunlight is less than 24 hours but greater than 21 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 21) <= sunlight_duration < datetime.timedelta(hours = 24):
        # (i) 'd_prime' is the duration of sunlight doubled minus 24 hours
    d_prime = 2 * sunlight_duration - datetime.timedelta(hours = 24)
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise = av_noon - (d_prime / 2)
        # (iii) The new sunset time is calculated as the average noon time plus half of the duration of sunlight
    new_sunset = av_noon + (d_prime / 2)
else:
    print("polar circle")
# -------------------------------------
# 9. Calculating the polar circle for tomorrow
    # a) If the duration of sunlight is less than 3 hours but greater than 1 second, the sunrise and sunset times are calculated as follows:
if datetime.timedelta(seconds = 1) <= sunlight_duration_t < datetime.timedelta(hours = 3):
        # (i) 'd_prime_t' is the duration of sunlight doubled
    d_prime_t = 2 * sunlight_duration_t
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise_t = av_noon - (d_prime_t / 2)

    # b) If the duration of sunlight is less than 9 hours but greater than 3 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 3) <= sunlight_duration_t < datetime.timedelta(hours = 9):
        # (i) 'd_prime_t' is the duration of sunlight plus 3 hours
    d_prime_t = sunlight_duration_t + datetime.timedelta(hours = 3)
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise_t = av_noon - (d_prime_t / 2)

    # c) If the duration of sunlight is less than 15 hours but greater than 9 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 9) <= sunlight_duration_t < datetime.timedelta(hours = 15):
        # (i) 'd_prime_t' is 12 hours
    d_prime_t = datetime.timedelta(hours = 12)
        # (ii) The new sunrise time is calculated as the average noon time minus half of the duration of sunlight
    new_sunrise_t = av_noon - (d_prime_t / 2)
    
    # d) If the duration of sunlight is less than 21 hours but greater than 15 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 15) <= sunlight_duration_t < datetime.timedelta(hours = 21):
        # (i) 'd_prime_t' is the duration of sunlight minus 3 hours
    d_prime_t = sunlight_duration_t - datetime.timedelta(hours = 3)
        # (ii) The new sunrise time is calculated as the average noon time minus half of d_prime_t
    new_sunrise_t = av_noon - (d_prime_t / 2)
    
    # e) If the duration of sunlight is less than 24 hours but greater than 21 hours, the sunrise and sunset times are calculated as follows:
elif sunlight_duration_t < datetime.timedelta(hours = 24):
        # (i) 'd_prime_t' is the duration of sunlight doubled minus 24 hours
    d_prime_t = 2 * sunlight_duration_t - datetime.timedelta(hours = 24)
        # (ii) The new sunrise time is calculated as the average noon time minus half of d_prime_t
    new_sunrise_t = av_noon - (d_prime_t / 2)
else:
    # f) If none of the above conditions are met, it is assumed that the location is in the polar circle.
        # In this case, the sunrise and sunset times are not calculated, and a message is printed.
    print("polar circle")

# -------------------------------------

# 10. Calculating the polar circle for yesterday
    # a) If the duration of sunlight is less than 3 hours but greater than 1 second, the sunrise and sunset times are calculated as follows:
if datetime.timedelta(seconds = 1) <= sunlight_duration_y < datetime.timedelta(hours = 3):
        # (i) 'd_prime_' is twice the duration of the afternoon
    d_prime_y = 2 * sunlight_duration_y
        # (ii) New sunset time 12pm + half of d_prime_y
    new_sunset_y = av_noon + (d_prime_y / 2)
    # b) If the duration of sunlight is less than 9 hours but greater than 3 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 3) <= sunlight_duration_y < datetime.timedelta(hours = 9):
        # (i) 'd_prime_y' is the duration of sunlight plus 3 hours
    d_prime_y = sunlight_duration_y + datetime.timedelta(hours = 3)
        # (ii) New sunset time 12pm + half of d_prime_y
    new_sunset_y = av_noon + (d_prime_y / 2)
    # c) If the duration of sunlight is less than 15 hours but greater than 9 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 9) <= sunlight_duration_y < datetime.timedelta(hours = 15):
        # (i) 'd_prime_y' is 12 hours
    d_prime_y = datetime.timedelta(hours = 12)
        # (ii) New sunset time 12pm + half of d_prime_y
    new_sunset_y = av_noon + (d_prime_y / 2)
    # d) If the duration of sunlight is less than 21 hours but greater than 15 hours, the sunrise and sunset times are calculated as follows:
elif datetime.timedelta(hours = 15) <= sunlight_duration_y < datetime.timedelta(hours = 21):
        # (i) 'd_prime_y' is the duration of sunlight minus 3 hours
    d_prime_y = sunlight_duration_y - datetime.timedelta(hours = 3)
        # (ii) New sunset time 12pm + half of d_prime_y
    new_sunset_y = av_noon + (d_prime_y / 2)
    # e) If the duration of sunlight is less than 24 hours but greater than 21 hours, the sunrise and sunset times are calculated as follows:
elif sunlight_duration_y < datetime.timedelta(hours = 24):
        # (i) 'd_prime_y' is the duration of sunlight doubled minus 24 hours
    d_prime_y = 2 * sunlight_duration_y - datetime.timedelta(hours = 24)
        # (ii) New sunset time 12pm + half of d_prime_y
    new_sunset_y = av_noon + (d_prime_y / 2)
else:
    print("polar circle")